# Harnessing Object and Scene Semantics for Large-Scale Video Understanding

[paper link](http://cs.brown.edu/~ls/Publications/cvpr2016wu.pdf)

## 动机

识别vedio中的动作（action）和场景语义（scene semantics）。

## 模型架构

包含了三种信息流的语义融合网络（semantic fusion network），分为三层：1.基于帧的低层次的CNN特征；2.基于对象（object）的大型CNN，用来识别对象；3.场景特征